In [1]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.pipeline import Pipeline
from pyspark.ml.clustering import KMeans, KMeansModel
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

from pymongo import MongoClient
import pandas as pd
import numpy as np
import json
import difflib
import tqdm
import time
from get_categories import get_categories
from get_transactions import get_transaction_data

transactions_df = get_transaction_data()
project_smc_df = get_categories(True)

transaction_project_df = pd.merge(transactions_df,
                                project_smc_df,
                                left_on='to_address', 
                                right_on='address', 
                                how='inner')

address_categories = transaction_project_df.groupby(["mapped_categories", "from_address"]).count().reset_index()[['mapped_categories', 'from_address', "to_address", '_id_x']]
address_categories.columns = ["categories", "userAddress", "projectAddress", "count"]

for cate in address_categories.categories.unique():
    address_categories[f'{cate}_count'] = len(address_categories[address_categories.categories == cate])
    
address_categories.to_csv("address_categories.csv", index=False)

f:\BTVN\HUST_doc\20231\BigData\Final_project\get_transactions.py:6: DtypeWarning: Columns (17,21,22,23,24,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  transactions = pd.read_csv("transactions.csv")


abc


f:\BTVN\HUST_doc\20231\BigData\Final_project\get_categories.py:9: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  smart_contracts_df = pd.read_csv("project_smc.csv")


Def


In [2]:
address_categories = transaction_project_df.groupby(["mapped_categories", "from_address"]).count().reset_index()[['mapped_categories', 'from_address', '_id_x']]
address_categories.columns = ["categories", "userAddress", "count"]

In [3]:
for cate in address_categories.categories.unique():
    address_categories[f'{cate}_count'] = len(address_categories[address_categories.categories == cate])
    
address_categories.to_csv("address_categories.csv", index=False)

In [4]:
address_categories.head()

,categories,userAddress,count,Art_count,Derivatives_count,Dexes_count,Gaming_count,Infrastructure_count,Lending_count,NFT_count,Services_count,Stablecoins_count,Virtual Worlds_count
0,Art,0x0012195f694f8125438c03c1692917e6de5fda76,1,2187,1535,348,115,23099,513,3272,1354,2644,809
1,Art,0x0021fedbd4d45d9762a2aadc4e4837ac949e8bb0,1,2187,1535,348,115,23099,513,3272,1354,2644,809
2,Art,0x00256d6f9757a193778c21eda9f8df4f74f47e7d,1,2187,1535,348,115,23099,513,3272,1354,2644,809
3,Art,0x002e1798bff1ea5bcd703133eb61706070080c19,1,2187,1535,348,115,23099,513,3272,1354,2644,809
4,Art,0x004b883f8adf1d0fabafc129e9ff9a43fea525fc,1,2187,1535,348,115,23099,513,3272,1354,2644,809


In [5]:
spark = SparkSession.builder.appName("test").getOrCreate()
df = spark.read.options(inferSchema='True',delimiter=',', header='True').csv("address_categories.csv")

feature_cols = df.columns
feature_cols.remove("categories")
feature_cols.remove("userAddress")

va = VectorAssembler(inputCols= feature_cols,
                    outputCol = "features")
kmeans = KMeans()

model = kmeans.fit(va.transform(df))
# pipeline = Pipeline(stages=[va, kmeans])

# pipeline_model = pipeline.fit(df)

In [6]:
# res = pipeline_model.transform(df).select("address", "prediction").toPandas()
centroids = model.clusterCenters()

distance_columns = [
    F.sqrt(sum((F.col(feature_cols[i]) - centroids[j][i]) ** 2 for i in range(len(feature_cols))))
    for j in range(len(centroids))
]

# Add a column 'distance_to_centroid' to the DataFrame
df_with_distance = model.transform(va.transform(df)).withColumn(
    "distance_to_centroid",
    distance_columns[model.transform(va.transform(df)).select("prediction").collect()[0]["prediction"]]
)


In [7]:
res_df = df_with_distance.select("userAddress", "prediction", "distance_to_centroid").toPandas()

In [ ]:
def infer(df):
    global pipeline_model, project_smc_df
    res: pd.DataFrame = pipeline_model.transform(df).select("address", "prediction").toPandas()
    
    

In [17]:
merged_df = pd.merge(res_df,
                     transaction_project_df[["from_address", "to_address"]],
                     left_on='userAddress',
                     right_on='from_address',
                     how='inner')

In [22]:
merged_df.columns

Index(['userAddress', 'prediction', 'distance_to_centroid', 'from_address',
       'to_address'],
      dtype='object')

In [21]:
merged_df.groupby("prediction")[['to_address', 'distance_to_centroid']].agg(list).reset_index()

,prediction,to_address,distance_to_centroid
0,0,"[0x95ad61b0a150d79219dcf64e1e6cc01f0b64c4ce, 0...","[0.49535681418890665, 0.49535681418890665, 0.4..."
1,1,"[0xfd086bc7cd5c481dcc9c85ebe478a1c0b69fcbb9, 0...","[173.5046431858111, 173.5046431858111, 173.504..."


In [16]:
transaction_project_df.columns

Index(['Unnamed: 0_x', '_id_x', 'type', 'hash', 'nonce', 'transaction_index',
       'from_address', 'to_address', 'value', 'gas', 'gas_price', 'input',
       'block_timestamp', 'block_number', 'block_hash',
       'receipt_cumulative_gas_used', 'receipt_gas_used',
       'receipt_contract_address', 'receipt_root', 'receipt_status',
       'item_timestamp', 'decoded_input_event', 'enriched_data', 'event_types',
       'related_addresses_event', 'decoded_input', 'related_addresses',
       'transaction_type', 'Unnamed: 0_y', '_id_y', 'address', 'name',
       'symbol', 'categories', 'chainId', 'mapped_categories'],
      dtype='object')